<a href="https://colab.research.google.com/github/yeipi-mora/personal-projects/blob/main/backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Paquetería**

In [ ]:
pip install backtrader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 7.4 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade matplotlib
!pip install --upgrade plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 99.4 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.1
    Uninstalling matplotlib-3.7.1:
      Successfully uninstalled matplotlib-3.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 99.6 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [ ]:
from google.colab import files
import datetime
import backtrader as bt
import pandas as pd
import shutil
import os
import matplotlib.pyplot as plt

# **Estrategias y Optimizacion**

In [ ]:
bb_params = {'period': range(10,61,5), 'devfactor': range(2,6)}

class PavoBB3(bt.Strategy):

    params = (
        ('period',20), # Tuple of tuples containing any variable settings required by the strategy.
        ('devfactor', 2),
        ('printlog',False), # Stop printing the log of the trading strategy
        ('doprint', False),
        ('name', 'BB')

    )

    def __init__(self):
        self.dataclose= self.datas[0].close    # Keep a reference to the "close" line in the data[0] dataseries
        self.order = None # Property to keep track of pending orders.  There are no orders when the strategy is initialized.
        self.buyprice = None
        self.buycomm = None

        self.bb = bt.indicators.BollingerBands(self.datas[0], period=self.params.period, devfactor=self.params.devfactor)

    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint: # Add if statement to only log of printlog or doprint is True
            dt = dt or self.datas[0].datetime.date(0)
            print('{0}, {1}'.format(dt.isoformat(),txt))

    def notify_order(self, order):
        # 1. If order is submitted/accepted, do nothing
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 2. If order is buy/sell executed, report price executed
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: {0:8.2f}, Size: {1:8.2f} Cost: {2:8.2f}, Comm: {3:8.2f}'.format(
                    order.executed.price,
                    order.executed.size,
                    order.executed.value,
                    order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log('SELL EXECUTED, {0:8.2f}, Size: {1:8.2f} Cost: {2:8.2f}, Comm{3:8.2f}'.format(
                    order.executed.price,
                    order.executed.size,
                    order.executed.value,
                    order.executed.comm))

            self.bar_executed = len(self) #when was trade executed
         # 3. If order is canceled/margin/rejected, report order canceled
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self,trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS {0:8.2f}, NET {1:8.2f}'.format(
            trade.pnl, trade.pnlcomm))

    def next(self):
        # Log the closing prices of the series from the reference
        if self.order: # check if order is pending, if so, then break out
            return

        # since there is no order pending, are we in the market?
        if not self.position: # not in the market
            if self.dataclose[0] > self.bb.lines.top[0]:
                self.log('BUY CREATE {0:8.2f}'.format(self.dataclose[0]))
                self.order = self.buy()
        else: # in the market
            if self.dataclose[0] < self.bb.lines.bot[0]:
                self.log('SELL CREATE, {0:8.2f}'.format(self.dataclose[0]))
                self.order = self.sell()

    def stop(self):
        self.log('Period: {0:8.2f} StdDev {1:8.2f} Ending Value: {2:8.2f}'.format(
            self.params.period,
            self.params.devfactor,
            self.broker.getvalue()),
            doprint=self.params.doprint)

    def notify_order(self, order):
        # 1. If order is submitted/accepted, do nothing
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 2. If order is buy/sell executed, report price executed
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: {0:8.2f}, Size: {1:8.2f} Cost: {2:8.2f}, Comm: {3:8.2f}'.format(
                    order.executed.price,
                    order.executed.size,
                    order.executed.value,
                    order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log('SELL EXECUTED, {0:8.2f}, Size: {1:8.2f} Cost: {2:8.2f}, Comm{3:8.2f}'.format(
                    order.executed.price,
                    order.executed.size,
                    order.executed.value,
                    order.executed.comm))

            self.bar_executed = len(self) #when was trade executed
         # 3. If order is canceled/margin/rejected, report order canceled
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

def optimization(strategy: bt.Strategy, data: pd.DataFrame, params: dict) -> pd.DataFrame:

    # Instancia de Backtrader
    cerebro = bt.Cerebro(optreturn=False)
    cerebro.adddata(bt.feeds.PandasData(dataname=data))

    # Optimización de la estrategia
    cerebro.optstrategy(strategy, **params)

    cerebro.broker.setcash(100000.0)
    cerebro.broker.setcommission(commission=0.0003)
    cerebro.addsizer(bt.sizers.FixedSize, stake=1)

    # Analizadores
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Minutes, compression=1, factor=365)
    cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.NoTimeFrame)
    cerebro.addanalyzer(bt.analyzers.DrawDown)

    # Después de la optimización, registra los resultados en una lista
    runs = cerebro.run()

    optimized_results = list()

    for run in runs:
        for strategy in  run:
            metrics = dict()
            metrics['strategy'] = str()
            for key, value in strategy.params.__dict__.items():
                if (type(value) is bool) or (type(value) is str): continue
                metrics[key] = value
                metrics['strategy'] += f'{value}_'
            metrics['strategy'] += strategy.params.name
            metrics['sharpe'] = strategy.analyzers.sharperatio.get_analysis()['sharperatio']
            metrics['return'] = '{0:.2f}%'.format(list(strategy.analyzers.timereturn.get_analysis().values())[0] * 100)
            metrics['drawdown'] = '{0:.2f}%'.format(strategy.analyzers.drawdown.get_analysis()['max']['drawdown'])
            optimized_results.append(metrics)

    df = pd.DataFrame(optimized_results)
    df.sort_values(by='return', ascending=False, inplace=True)
    df.set_index('strategy', inplace=True)

    return df

# **Mejor Rendimiento a Hoy**

In [ ]:
if os.path.exists('/content/data.csv'):
  os.remove('/content/data.csv')
uploaded = files.upload()

Saving data.csv to data.csv


In [ ]:
previous_data = pd.read_csv('data.csv')
previous_data['timestamp'] = pd.to_datetime(previous_data['timestamp'])  # Convierte la columna en objetos de fecha y hora de Pandas
previous_data.set_index('timestamp', inplace=True)

df = optimization(PavoBB3, previous_data, params=bb_params)

print(df.head())
period, devfactor = int(df['period'][0]), int(df['devfactor'][0])

          period  devfactor    sharpe return drawdown
strategy                                             
50_3_BB       50          3  0.057236  7.33%    1.28%
30_2_BB       30          2  0.057260  7.19%    1.66%
35_3_BB       35          3  0.055436  7.16%    1.28%
40_3_BB       40          3  0.055436  7.16%    1.28%
60_3_BB       60          3  0.055421  7.14%    1.28%


# **Rendimiento de Estrategia Por Mes**

In [ ]:
# Verifica si el archivo existe antes de intentar eliminarlo
for i in range(12):
  if os.path.exists('/content/data'+str(i)+'.csv'):
      os.remove('/content/data'+str(i)+'.csv')
uploaded = files.upload()

Saving data1.csv to data1.csv
Saving data2.csv to data2.csv
Saving data3.csv to data3.csv
Saving data4.csv to data4.csv
Saving data5.csv to data5.csv
Saving data6.csv to data6.csv
Saving data7.csv to data7.csv
Saving data8.csv to data8.csv
Saving data9.csv to data9.csv
Saving data10.csv to data10.csv


In [ ]:
if __name__ == '__main__':

  parameters = list()

  for i in range(11):
    try:
      if os.path.exists('/content/data'+str(i)+'.csv'):

        previous_data = pd.read_csv('data'+str(i)+'.csv')
        previous_data['timestamp'] = pd.to_datetime(previous_data['timestamp'])  # Convierte la columna en objetos de fecha y hora de Pandas
        previous_data.set_index('timestamp', inplace=True)

        df = optimization(PavoBB3, previous_data, params=bb_params)

        #print(df.head())
        period, devfactor = int(df['period'][0]), int(df['devfactor'][0])

        parameters.append([period, devfactor])
    except: continue

In [ ]:
for i in range(11):
    try:
      period = parameters[i][0]
      devfactor = parameters[i][1]
      data = pd.read_csv('data'+str(i+1)+'.csv')
      data['timestamp'] = pd.to_datetime(data['timestamp'])  # Convierte la columna en objetos de fecha y hora de Pandas
      data.set_index('timestamp', inplace=True)

      # Instancia de Backtrader
      cerebro = bt.Cerebro()
      cerebro.adddata(bt.feeds.PandasData(dataname=data))
      cerebro.addstrategy(PavoBB3, period=period, devfactor=devfactor)

      cerebro.broker.setcash(100000.0)
      cerebro.broker.setcommission(commission=0.0003)
      cerebro.addsizer(bt.sizers.FixedSize, stake=1)

      # Analizadores
      cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Minutes, compression=1, factor=365)
      cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.NoTimeFrame)
      cerebro.addanalyzer(bt.analyzers.DrawDown)
      cerebro.addanalyzer(bt.analyzers.TradeAnalyzer)

      strategy = cerebro.run()[0]

      sharpe = strategy.analyzers.sharperatio.get_analysis()['sharperatio']
      ret = '{0:.2f}%'.format(list(strategy.analyzers.timereturn.get_analysis().values())[0] * 100)
      drawdown = '{0:.2f}%'.format(strategy.analyzers.drawdown.get_analysis()['max']['drawdown'])

      # Accede a los resultados del TimeReturn
      trade_analyzer = strategy.analyzers.tradeanalyzer.get_analysis()
      print(f'\n***********MES {i}***********\n')
      print(f'Parametros: {(period, devfactor)}\n')
      try: print(f'Sharpe Ratio: {round(sharpe,4)}\n')
      except: print('Error en el Sharpe Ratio\n')

      try: print(f'Return: {ret}\n')
      except: print('Error en Return\n')

      try: print(f'Max DrawDown: {drawdown}\n')
      except: print('Error en el MaxDrawDown\n')

      try: print('Total de operaciones:', trade_analyzer['total']['total'])
      except: print('Error en el numero de operacions\n')

      try: print('\nTotal de operaciones fallidas:', trade_analyzer['lost']['total'])
      except: print('\nError en el numero de operaciones fallidas\n')

    except: continue


***********MES 0***********

Parametros: (60, 3)

Sharpe Ratio: 0.0567

Return: 6.16%

Max DrawDown: 1.13%

Total de operaciones: 1

Error en el numero de operaciones fallidas


***********MES 1***********

Parametros: (55, 3)

Sharpe Ratio: -0.0031

Return: 1.78%

Max DrawDown: 1.52%

Total de operaciones: 2

Total de operaciones fallidas: 0

***********MES 2***********

Parametros: (25, 4)

Sharpe Ratio: 0.0415

Return: 5.80%

Max DrawDown: 1.86%

Total de operaciones: 1

Total de operaciones fallidas: 0

***********MES 3***********

Parametros: (60, 3)

Sharpe Ratio: -0.0002

Return: 1.90%

Max DrawDown: 2.00%

Total de operaciones: 2

Total de operaciones fallidas: 0

***********MES 4***********

Parametros: (55, 3)

Sharpe Ratio: -0.0422

Return: 0.05%

Max DrawDown: 1.37%

Total de operaciones: 1

Total de operaciones fallidas: 0

***********MES 5***********

Parametros: (20, 3)

Sharpe Ratio: 0.0372

Return: 4.73%

Max DrawDown: 1.45%

Total de operaciones: 2

Total de operacio